In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rioxarray
import xarray as xr
import warnings
from scipy.interpolate import make_interp_spline
from typing import Callable, Union

## Analysis NUTS dataset

In [ ]:
!pwd

In [ ]:
# Set filepaths
nuts_filepath = "../../data/in/NUTS_RG_10M_2024_4326.shp/NUTS_RG_10M_2024_4326.shp"
era5_filepath = "../../data/in/ERA5land_global_t2m_dailyStats_mean_01Deg_2024_08_data.nc"

In [ ]:
# --- 1. Load NetCDF temperature data ---
era5_ds = xr.open_dataset(era5_filepath)

In [ ]:
t2m = era5_ds["t2m"].isel(valid_time=0)

plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
t2m.plot(ax=ax, transform=ccrs.PlateCarree(), cmap="coolwarm")
ax.coastlines()
ax.set_title("T2M - Global")
plt.show()

In [ ]:
# --- 2. Load NUTS shapefile ---
nuts_gdf = gpd.read_file(nuts_filepath)

In [ ]:
# initialize an empty figure and add an axis
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot()

# plot a basic map of the world
nuts_gdf.plot(ax=ax, color="lightgray", edgecolor="black", alpha=0.5)

# turn off axis ticks
ax.set_xticks([])
ax.set_yticks([])

# set the plot title
plt.title("Basic Map of World with GeoPandas")
plt.show()

In [ ]:
# Bounding Box defining EUROPA
europe_bounds_manual = {
    "min_longitude": -10,
    "min_latitude": 35,
    "max_longitude": 40,
    "max_latitude": 72,
}

# Bounding Box defining EUROPA
minx, miny, maxx, maxy = nuts_gdf.total_bounds
europe_bounds_nuts = {
    "min_longitude": minx,
    "min_latitude": miny,
    "max_longitude": maxx,
    "max_latitude": maxy,
}

# bounding_box = europe_bounds_manual
bounding_box = europe_bounds_nuts

## Cropping NETCDF4 data

In [ ]:
lat = era5_ds.latitude.values
is_lat_descending = lat[0] > lat[-1]
print(f"Descending latitude: {is_lat_descending}")

lat_slice = (
    slice(bounding_box["max_latitude"], bounding_box["min_latitude"])
    if is_lat_descending
    else slice(bounding_box["min_latitude"], bounding_box["max_latitude"])
)

lon_slice = slice(bounding_box["min_longitude"], bounding_box["max_longitude"])

cropped_ds = era5_ds.sel(latitude=lat_slice, longitude=lon_slice)

## Cropping NUTS DATA

In [ ]:
nuts_gdf = nuts_gdf[
    (nuts_gdf.geometry.bounds["minx"] >= bounding_box["min_longitude"])
    & (nuts_gdf.geometry.bounds["miny"] >= bounding_box["min_latitude"])
    & (nuts_gdf.geometry.bounds["maxx"] <= bounding_box["max_longitude"])
    & (nuts_gdf.geometry.bounds["maxy"] <= bounding_box["max_latitude"])
]

## Plot data in the same map

In [ ]:
t2m = cropped_ds["t2m"].isel(valid_time=0)

# Ensure the same CRS
nuts_gdf = nuts_gdf.to_crs("EPSG:4326")

# --- 3. Plot both together ---
plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# Plot temperature raster (use .plot() directly from xarray)
t2m.plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap="coolwarm",
    cbar_kwargs={"label": "Temperature (Celsius)"},
)

# Plot NUTS regions
nuts_gdf.boundary.plot(ax=ax, edgecolor="black", linewidth=0.5)

# Add base map decorations
ax.set_title("ERA5 T2M and NUTS3 Regions - August 2024", fontsize=14)
ax.coastlines()
ax.gridlines(draw_labels=True)

plt.show()

## Experiments Interpolation functions

In [ ]:
def linear_interpolator(x_data, y_data, x_new):
    """Wrap a linear interpolation function."""
    try:
        result = np.interp(x_new, x_data, y_data)
    except Exception as e:
        print(f"Caught exception: {type(e).__name__} - {e}")
        raise  # re-raise the exception if you want it propagated
    return result

In [ ]:
def bspline_interpolator(x_data, y_data, x_new, degree=1):
    """Wrap a B-spline interpolation function."""
    try:
        bspl = make_interp_spline(x_data, y_data, k=degree)
        result = bspl(x_new)
    except Exception as e:
        print(f"Caught exception: {type(e).__name__} - {e}")
        raise  # re-raise the exception if you want it propagated
    return result

In [ ]:
def interpolator_function(
    x_data: np.ndarray,
    y_data: np.ndarray,
    method: str = "linear",
    b_spline_degree: int = 3,
) -> Callable[[Union[float, np.ndarray]], np.ndarray]:
    """
    Create an interpolation function using specified method.

    Parameters
    ----------
    x_data : array-like, shape (n,)
        Independent variable data points (must be sorted ascending).
    y_data : array-like, shape (n,)
        Dependent variable data points.
    method : str, optional
        Interpolation method: "linear" or "bspline" (default "linear").
    b_spline_degree : int, optional
        Degree of B-spline (default 3). Used only if method="bspline".

    Returns
    -------
    interpolator : function
        Function accepting scalar or array x_new and returning interpolated y values.

    Raises
    ------
    ValueError
        If inputs are invalid or unsupported method is specified.
    """

    x_data = np.asarray(x_data)
    y_data = np.asarray(y_data)

    if x_data.ndim != 1 or y_data.ndim != 1:
        raise ValueError("x_data and y_data must be one-dimensional arrays.")
    if len(x_data) != len(y_data):
        raise ValueError("x_data and y_data must have the same length.")
    if len(x_data) < 2:
        raise ValueError("At least two data points are required for interpolation.")

    # Check sortedness
    if not np.all(np.diff(x_data) >= 0):
        raise ValueError("x_data must be sorted in ascending order.")

    if method not in {"linear", "bspline"}:
        raise ValueError(
            f"Unsupported interpolation method '{method}'. Choose 'linear' or 'bspline'."
        )

    if method == "bspline":
        if not isinstance(b_spline_degree, int) or not (1 <= b_spline_degree <= 5):
            raise ValueError("b_spline_degree must be an integer between 1 and 5.")
        try:
            spline_obj = make_interp_spline(x_data, y_data, k=b_spline_degree)
        except Exception as e:
            raise RuntimeError(f"Failed to create spline: {e}") from e

        def bspline_interpolator(x_new):
            return spline_obj(x_new)

    def linear_interpolator(x_new):
        return np.interp(x_new, x_data, y_data)

    return bspline_interpolator if method == "bspline" else linear_interpolator

In [ ]:
# Ground Truth curve
x_ground_truth = np.linspace(-6, 6, 50)
y_ground_truth = np.power(x_ground_truth, 3)

# Data points
x_data = np.arange(-4, 5, 2)
y_data = np.power(x_data, 3)

# Data for evaluation
x_test = np.linspace(-6, 6, 20)
y_test = linear_interpolator(x_data, y_data, x_new=x_test)

plt.plot(x_ground_truth, y_ground_truth, "--", label="Ground Truth", color="blue")
plt.plot(x_data, y_data, "D", label="Data", color="orange")
plt.plot(x_test, y_test, "-*", label="Linear Interpolation", color="green")

# Shade regions where data is missing (e.g., x < 4 and x > 4)
plt.axvspan(min(x_data), -7, color="gray", alpha=0.3)
plt.axvspan(7, max(x_data), color="gray", alpha=0.3, label="No Data")

plt.xlim(-7, 7)  # Set x-axis limits from 0 to 4
plt.ylim(-100, 100)  # Set y-axis limits from 3 to 7
plt.legend(loc="best")
plt.grid()
plt.show()

In [ ]:
# Ground Truth curve
x_ground_truth = np.linspace(-6, 6, 50)
y_ground_truth = np.power(x_ground_truth, 3)

# Data points
x_data = np.arange(-4, 5, 2)
y_data = np.power(x_data, 3)

# Data for evaluation
x_test = np.linspace(-6, 6, 20)
y_test = bspline_interpolator(x_data, y_data, x_new=x_test, degree=3)

plt.plot(x_ground_truth, y_ground_truth, "--", label="Ground Truth", color="blue")
plt.plot(x_data, y_data, "D", label="Data", color="orange")
plt.plot(x_test, y_test, "-*", label="Linear Interpolation", color="green")

# Shade regions where data is missing (e.g., x < 4 and x > 4)
plt.axvspan(min(x_data), -7, color="gray", alpha=0.3)
plt.axvspan(7, max(x_data), color="gray", alpha=0.3, label="No Data")

plt.xlim(-7, 7)  # Set x-axis limits from 0 to 4
plt.ylim(-100, 100)  # Set y-axis limits from 3 to 7
plt.legend(loc="best")
plt.grid()
plt.show()

In [ ]:
li = interpolator_function(x_data, y_data, method="linear")

In [ ]:
from scipy.interpolate import make_interp_spline

# Ground Truth curve
x_ground_truth = np.linspace(-6, 6, 50)
y_ground_truth = np.power(x_ground_truth, 3)

# Data points
x_data = np.arange(-4, 5, 2)
y_data = np.power(x_data, 3)

# Data for evaluation
x_test = np.linspace(-6, 6, 20)
interp_fun = interpolator_function(x_data, y_data, method="bspline", b_spline_degree=1)
y_test = interp_fun(x_test)

plt.plot(x_ground_truth, y_ground_truth, "--", label="Ground Truth", color="blue")
plt.plot(x_data, y_data, "D", label="Data", color="orange")
plt.plot(x_test, y_test, "-*", label="Linear Interpolation", color="green")

# Shade regions where data is missing (e.g., x < 4 and x > 4)
plt.axvspan(min(x_data), -7, color="gray", alpha=0.3)
plt.axvspan(7, max(x_data), color="gray", alpha=0.3, label="No Data")

plt.xlim(-7, 7)  # Set x-axis limits from 0 to 4
plt.ylim(-100, 100)  # Set y-axis limits from 3 to 7
plt.legend(loc="best")
plt.grid()
plt.show()

In [ ]:
era5_ds["valid_time"]

In [ ]:
# temperature_at_point = era5_ds.t2m.sel(
#    latitude=-10,
#    longitude=35,
#    valid_time="2024-08-01",
#    method="nearest",  # Finds the closest grid point
# ).item()  # .item()

# temperature_at_point

In [ ]:
# era5_ds["t2m"] = era5_ds["t2m"] * 2

In [ ]:
# temperature_at_point1 = era5_ds.t2m.sel(
#    latitude=-10,
#    longitude=35,
#    valid_time="2024-08-01",
#    method="nearest",  # Finds the closest grid point
# ).item()  # .item()

# temperature_at_point1

In [ ]:
# era5_ds["t2m"][0, :, :].shape

In [ ]:
from R0_pip_fun import R0_pip_fun

In [ ]:
era5_ds["t2m"] = xr.DataArray(
    R0_pip_fun(era5_ds["t2m"].values),
    dims=era5_ds["t2m"].dims,
    coords=era5_ds["t2m"].coords,
    attrs=era5_ds["t2m"].attrs,
)

In [ ]:
era5_ds["t2m"].dims

In [ ]:
era5_ds["t2m"].attrs

In [ ]:
numpy_array = era5_ds["t2m"].values

In [ ]:
non_nan_values = numpy_array[~np.isnan(numpy_array)]

In [ ]:
non_nan_values

In [ ]:
def create_tiny_netcdf(filename="tiny_temp_data.nc"):
    """
    Creates a tiny NetCDF file with 'temperature', 'latitude', 'longitude',
    and 'valid_time' variables, including some NaN values for interpolation testing.

    Args:
        filename (str): The name of the NetCDF file to create.
    """
    # 1. Define simple coordinates
    times = pd.date_range("2024-01-01", periods=3, freq="12H")  # 3 time points
    latitudes = np.array([40.0, 40.5, 41.0])  # 3 latitude points
    longitudes = np.array([10.0, 10.5, 11.0])  # 3 longitude points

    # 2. Create a small NumPy array for temperature data
    #    Shape: (time, lat, lon) = (3, 3, 3)
    #    Introduce some NaNs intentionally for testing interpolation of missing values
    temp_data = np.array(
        [
            [
                [0, 0.85, 2.35],
                [1.35, np.nan, 2.85],
                [1.85, 3.35, 3.85],
            ],
            [
                [2.85, np.nan, 4.85],
                [4.35, 4.85, 5.85],
                [5.35, 6.35, np.nan],
            ],  # NaN at this point
            [
                [6.85, 7.85, 8.85],
                [7.35, 8.35, 9.35],
                [7.85, 8.85, 9.85],
            ],
        ]
    )

    # 3. Create an xarray DataArray for temperature
    temperature_da = xr.DataArray(
        temp_data,
        coords={"valid_time": times, "latitude": latitudes, "longitude": longitudes},
        dims=["valid_time", "latitude", "longitude"],
        name="t2m",
        attrs={"units": "Celsius", "long_name": "Air Temperature at Surface"},
    )

    # 4. Create an xarray Dataset from the DataArray
    ds = xr.Dataset({"temperature": temperature_da})

    # Add some global attributes (optional)
    ds.attrs["description"] = "Tiny NetCDF file for interpolation testing"
    ds.attrs["created_by"] = "Gemini LLM"

    # 5. Save the xarray Dataset to a .nc file
    ds.to_netcdf(filename)
    print(f"Tiny NetCDF file '{filename}' created successfully in the current directory.")
    print("\nDataset structure:")
    print(ds)

In [ ]:
create_tiny_netcdf()

In [ ]:
# ds = xr.open_dataset("tiny_temp_data.nc")
ds = xr.open_dataset(era5_filepath)

In [ ]:
ds

In [ ]:
ds.t2m

In [ ]:
ds.rio.write_crs("EPSG:4326", inplace=True)

In [ ]:
# Set spatial dimensions (if not automatically recognized, though rioxarray is good at this)
ds = ds.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True)

ds = ds.rename({"valid_time": "band"})

In [ ]:
# Save the first time slice of the temperature DataArray as a GeoTIFF
output_filename = "my_xarray_data.tif"
ds.t2m.rio.to_raster(output_filename)

In [ ]:
# Specify the path to your .tif file
file_path = output_filename  # Replace with the actual path to your .tif file

# Open the .tif file as an xarray DataArray
try:
    raster_data = rioxarray.open_rasterio(file_path)
    print("Successfully opened the .tif file:")
    print(raster_data)
except Exception as e:
    print(f"Error opening the .tif file: {e}")

In [ ]:
try:
    verify_raster = rioxarray.open_rasterio(output_filename)
    print("\nSuccessfully re-opened the saved multi-band raster for verification:")
    print(verify_raster)
    # You can plot individual bands or all bands if matplotlib supports it
    # import matplotlib.pyplot as plt
    plt.figure(figsize=(12, 6))
    # Plot the first band (first time step)
    verify_raster.isel(band=0).plot(
        ax=plt.subplot(1, 2, 1),
        cmap="viridis",
        cbar_kwargs={"label": "Processed Temperature (Band 0)"},
    )
    plt.title("First Time Step/Band")
    # Plot the second band (second time step)
    verify_raster.isel(band=1).plot(
        ax=plt.subplot(1, 2, 2),
        cmap="viridis",
        cbar_kwargs={"label": "Processed Temperature (Band 1)"},
    )
    plt.title("Second Time Step/Band")
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"\nError verifying saved raster: {e}")

In [ ]:
ds.dims